In [1]:
pip install wordninja

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import wordninja

####### After importing nltk, run the following only once ######
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
###pip install wordninja ###

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MYPC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MYPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MYPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def remove_htmlcodes(document):
    replacement = {
                    "&ampnbsp": ' ',
                    "&ampamp": '&',
                    "&ampquot": '\'',
                    "&ampldquo": '\"',
                    "&amprdquo": '\"',
                    "&amplsquo": '\'',
                    "&amprsquo": '\'',
                    "&amphellip": '...',
                    "&ampndash": '-',
                    "&ampmdash": '-'
                  }
    for str in replacement:
        document = document.replace(str, replacement[str])
        
    return document

In [4]:
def get_wordnet_pos(word):
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J": wordnet.ADJ, 
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemma_stop(str):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer('\w+|\$]\d\[+|\S+,-')
    tokenized = tokenizer.tokenize(str)
    lemmatized = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokenized]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in lemmatized if w.lower() not in stop_words]
    after_lemma_stop = ' '.join(w for w in filtered_sentence)
    return filtered_sentence

In [6]:
data = pd.read_csv("sports.csv")
data = np.array(data)
np.save("dataset",data)
data = np.load("dataset.npy",allow_pickle = True) 

In [7]:
get_docID = {}
get_index = {}


NN = len(data)
for i in range(0, len(data)) :
    get_docID[i] = data[i][0]
    get_index[data[i][0]] = i

In [58]:
def is_not_credible (text):
    match = re.search(r'[!@#?&{}()]', text)
    
    if match:
        return True
    else:
        return False

In [59]:
def scrub_words(text):
    text = re.sub('[!@#?&{}()]', '', text)
    text=re.sub(r'[^\x00-\x7F]'," ",text)
    return text

In [60]:
def clean_document (document_string):
    cleaned_doc = document_string
    for word in document_string.split():
                if is_not_credible(word):
                    temp= scrub_words(word)
                    split=wordninja.split(temp)
                    if len(split)>7:
                          cleaned_doc = cleaned_doc.replace(word,'')
    return cleaned_doc

In [61]:
from datetime import datetime
def replace_dates(documentString):
    
    regEx = '(([0-9]+(/|\\.|-)[0-9]+(/|\\.|-)[0-9]+)|([0-9]+(/|\\.|-)[0-9]+))'
    iterator = re.finditer(regEx, documentString)
    listOfDates = [(m.start(0), m.end(0)) for m in iterator]
    
    for indices in listOfDates:
        date = documentString[indices[0]:indices[1]]
        tmp = date
        date = date.replace('.', '/')
        date = date.replace('-', '/')
        count = date.count('/')
        newDate = ''
        if count == 2:
            try:
                newDate = datetime.strptime(date, '%m/%d/%Y').strftime('%d %b %Y')
            except ValueError as ve:
                newDate = date
        else:
            try:
                newDate = datetime.strptime(date, '%m/%d').strftime('%d %b')
            except ValueError as ve:
                newDate = date
                
        newDate = newDate.replace(' ', '')
        documentString = documentString.replace(tmp, newDate)
        # print(newDate)
    
    return documentString

*Run the following cell once after all pre-processing (removing JSON etc), and store final lemmatized contents of all docs:* 

In [65]:
# creating a temporary smaller dataset

subset = []
counter = 0
for document in data:
    subset.append(document)
    counter += 1
    if counter == NN:
        break
print(len(subset))

59575


In [66]:
import time
from tqdm import tqdm

start = time.time()

titles = []
contents = []
for document in tqdm(subset):
    # actually modifying the document
    document[4] = remove_htmlcodes(document[4])
    
    # not actually modifying the document
    modifiedContent = replace_dates(document[4])
    modifiedContent = lemma_stop(clean_document(modifiedContent))
    modifiedTitle = lemma_stop(clean_document(document[2]))
    
    # case-folding
    for i in range(len(modifiedContent)):
        modifiedContent[i] = modifiedContent[i].lower()
    
    # modifiedTitle = lemma_stop((document[1]))
    titles.append(modifiedTitle)
    contents.append(modifiedContent)
    
print(time.time() - start)  # 110.26236414909363

100%|██████████| 59575/59575 [50:27<00:00, 19.67it/s]  

3028.0473091602325


In [67]:
import unidecode
contents_temp = contents

titles_temp = titles

for i in range(NN):
    for j in range(len(contents[i])):
        contents[i][j] = unidecode.unidecode(contents[i][j])
    for j in range(len(titles[i])):
        titles[i][j] = unidecode.unidecode(titles[i][j])


In [68]:
import trie
import pickle 
# Create map from docID of the document to an object of class Node 
# (i.e, the corresponding document trie structure)
# ex. if the docID of the document is 1, 
# getReference[1] gives the object which is the trie structure of docID 1

getReference = {}

In [69]:
documentRoot = []
collection = trie.CollectionNode()



# initializing the root for 1000 documents
for i in range(NN):
    newDocument = trie.Node()
    documentRoot.append(newDocument)
    getReference[get_docID[i]] = newDocument

In [70]:
# creating the documents

max_tf = {}

import time
from tqdm import tqdm

start = time.time()
for i in tqdm(range(NN)):
    for w in contents_temp[i]:
        collection.add_document(w, 0, get_docID[i])
        documentRoot[i].add(w, 0)
        if get_docID[i] in max_tf:
            max_tf[get_docID[i]] = max(documentRoot[i].count_words(w, 0), max_tf[get_docID[i]])
        else:
            max_tf[get_docID[i]] = documentRoot[i].count_words(w, 0)
    for w in titles_temp[i]:
        collection.add_title(w, 0, get_docID[i])
        
print(time.time() - start)  #39.19705152511597

100%|██████████| 59575/59575 [01:22<00:00, 722.18it/s] 

82.5031087398529


In [71]:
import math
import queue

documentLength = {}
N = len(documentRoot)
print("hello")
print(N)

for i in tqdm(range(len(documentRoot))):
    
    docID = get_docID[i]
    length = 0
    document = documentRoot[i]
    q = queue.Queue()
    q.put([document, ''])
    
    while q.qsize() > 0:

        current = q.get()
        reference = current[0]
        word = current[1]

        if reference.words > 0:
            df = len(collection.get_doc_list(word, 0))
            idf = math.log10(N/df)
            # print(word, reference.words, df)
            length += (reference.words * idf) ** 2

        for i in range(256):
            if reference.children.get(i) is not None:
                new_word = word + chr(i)
                q.put([reference.children[i], new_word])

    # print(length**0.5)
    documentLength[docID] = length**0.5
    
    
    
pickle_out = open("collection.pickle","wb")
pickle.dump(collection,pickle_out)
pickle_out.close()
pickle_out1 = open("documentRoot.pickle","wb")
pickle.dump(getReference,pickle_out1)
pickle_out1.close()
pickle_out2 = open("max_tf.pickle","wb")
pickle.dump(max_tf,pickle_out2)
pickle_out2.close()



  0%|          | 8/59575 [00:00<13:18, 74.62it/s]

hello
59575


100%|██████████| 59575/59575 [12:03<00:00, 82.40it/s] 


In [72]:
import sys
print(sys.getsizeof(documentRoot))

514560


In [77]:
query = input()
final_query = replace_dates(query)
final_query = lemma_stop(final_query)
for i in range(len(final_query)):
    final_query[i] = unidecode.unidecode(final_query[i])
    
    # case-folding
    final_query[i] = final_query[i].lower()
print(final_query)
print(len(final_query))

Tennis
['tennis']
1


In [78]:
tf_query = {}
for w in final_query:
    if w not in tf_query:
        tf_query[w] = 1
    else:
        tf_query[w] += 1
   


***Ranked Retrieval based on TF-IDF Score :***


In [81]:
# scores[i] stores the dot product of the tf-idf score vectors of the query and document of docID i in the corpus
scores = {}
title_score = {}

# N is the total number of documents in the corpus
N = len(documentRoot)

# wordsInDoc[i] is a sorted list of (word, score) tuples where
# score is the tf-idf score for the (word, <ith doc>) pair
wordsInDoc = {}

factor = {}

import math
import bisect

for query_term in tf_query:
    
    docs_having_query_term = collection.get_doc_list(query_term, 0)
    
    df = len(docs_having_query_term)
    idf = 0
    
    print('-------------------------------------')
    print('Term in query = ', query_term)
    # print('List of documents with this term=', docs_having_query_term)
    print()
    
    if df == 0:
        idf = 0
    else:
        idf = math.log10(N/df)
        
    docs_having_query_term_in_title = collection.get_title_list(query_term,0)
    print(docs_having_query_term_in_title)
    for docID in docs_having_query_term_in_title:
        if docID in title_score:
            title_score[docID] += idf
        else:
            title_score[docID] = idf
        
#     print('df = ',df)
#     print('idf = ',idf)
    
    tfidf_query = tf_query[query_term] * idf
        
    for docID in docs_having_query_term:
        # print(docID)
        tf_doc = getReference[docID].count_words(query_term, 0)
        tf_doc = 0.5 + 0.5*tf_doc/max_tf[docID]
        # print('tf for doc',docID,'is',tf_doc)
        # tfidf_doc_query = tf_doc * idf
        # tfidf_doc = 1 + math.log10(tf_doc)
        tfidf_doc = (tf_doc)
        # tfidf_doc_query = (tf_doc)
        
        # print('tfidf for doc',doc,'is',tfidf_doc)
        # print()
        
        if docID not in scores:
            scores[docID] = (tfidf_query * tfidf_doc)
            wordsInDoc[docID] = []
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] = idf
        else:
            scores[docID] += (tfidf_query * tfidf_doc)
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] += idf
print(title_score)
for docID in scores:
    if documentLength[docID] != 0:
#         scores[docID] = scores[docID]/ math.sqrt(documentLength[docID])
        scores[docID] *= factor[docID]
        if docID in title_score:
            scores[docID] *= 1 + title_score[docID]


sorted_scores = sorted(scores.items(), key = lambda kv : kv[1] , reverse = True)

maxshow = min(10, len(scores))

print('\n\n')
print('===========================================================================================================')

for i in range(maxshow):
    # print(i)
    print()
    docID = sorted_scores[i][0]
    print('doc ID = ', docID)
    cnt = 0
    print('Keywords:')
    print()
    print(subset[get_index[sorted_scores[i][0]]][2])
    print()
    if sorted_scores[i][0] not in title_score:
        print('title score = ',0)
    else:
        print('title score = ',title_score[sorted_scores[i][0]])
    for j in range(len(wordsInDoc[docID])):
        print(wordsInDoc[docID][j][1], wordsInDoc[docID][j][0], end = ' ')
        print(getReference[docID].count_words(wordsInDoc[docID][j][1], 0))
    print()
    print()
    count = 0
    found = 0
    words_before=queue.Queue()
    at_start = 1
    display = ""
    for word in subset[get_index[docID]][4].split():
            
        check_with=replace_dates(word)
        check_with = check_with.lower()
        if len(lemma_stop(check_with)) > 0:
            check_with=lemma_stop(check_with)[0]
        else:
            check_with=word
        
        if check_with == wordsInDoc[docID][0][1]:
            found=1
            
        if found == 1:
            display = display + word + " "
            count += 1
            if count == 50:
                break
        if found == 0:
            words_before.put(word)
            if words_before.qsize()>20:
                remove=words_before.get()
                at_start=0
                
    if not at_start:
        print('...', end = ' ')
    while words_before.qsize() > 0:
        print(words_before.get(), end = ' ')
    print(display, end = ' ')
    print('...', end = ' ')
    print('\n')
    print('tf-idf score=', sorted_scores[i][1])
    print('\n')
    print('=======================================================================================================')

-------------------------------------
Term in query =  tennis

{'1607985258-59209', '1607985243-59133', '1607985163-58710', '1607985131-58535', '1607984673-56153', '1607984082-53249', '1607984294-54221', '1607985276-59317', '1607984930-57502', '1607984440-54992', '1607983918-52480', '1607984294-54219', '1607984731-56453', '1607985072-58213', '1607984510-55353', '1607985272-59276', '1607985122-58484', '1607984230-53876', '1607985016-57940', '1607985021-57974', '1607984247-53958', '1607985258-59210', '1607985317-59514', '1607985182-58813', '1607983853-52163', '1607985285-59365', '1607985268-59251', '1607984996-57843', '1607985211-58969', '1607985040-58052', '1607985086-58283', '1607985272-59300', '1607984266-54064', '1607984920-57460', '1607962354-10075', '1607985312-59478', '1607984179-53668', '1607984950-57607', '1607985272-59277', '1607984477-55200', '1607984066-53158', '1607985268-59261', '1607984632-55965', '1607984977-57734', '1607984033-53038', '1607984321-54359', '1607985224-5903

Board of governing body the International Tennis Federation wants to cut men's singles matches in the Davis Cup team competition to best-of-three sets, and members will be asked to approve the change in August.  ... 

tf-idf score= 11.40438487778659



doc ID =  1607984308-54292
Keywords:

India’s Somdev Devvarman announces retirement from pro tennis

title score =  1.9621506943756382
tennis -1.9621506943756382 1


Somdev Devvarman announces retirement from pro tennis after the 31-year old decided to give Chennai Open a miss.  ... 

tf-idf score= 11.40438487778659



doc ID =  1607985258-59203
Keywords:

Small exhibition events testing the waters as tennis plots way back onto court

title score =  1.9621506943756382
tennis -1.9621506943756382 2


The BASE Tennis Academy in the town of Höhr-Grenzhausen hosted a four-day Tennis Point Exhibition Series – an eight-man event.  ... 

tf-idf score= 11.40438487778659



doc ID =  1607984046-53079
Keywords:

Chris Evert stunned by Maria Sharapo